# Code for competition

In [1]:
!apt -y install --allow-change-held-packages libcudnn8=8.6.0.163-1+cuda11.8

!pip uninstall -y tensorflow 
#!pip uninstall -y tensorflow-transform
#!pip uninstall -y tensorflow-io
#!pip install tensorflow-transform
!pip install tensorflow

import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import shutil
from collections import Counter
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from PIL import Image
import re
import time
from IPython.display import FileLink

!nvidia-smi

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)
print(tf.config.list_physical_devices())

# Enable experimental feature of memory occupation growth control 
physical_devices = tf.config.experimental.list_physical_devices('GPU')
for dev in physical_devices:
    tf.config.experimental.set_memory_growth(dev, True)
    
# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')

# Enable distributed training
strategy = tf.distribute.MirroredStrategy()

# Random seed for reproducibility
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8 libcudnn8-dev
2 upgraded, 0 newly installed, 0 to remove and 91 not upgraded.
Need to get 883 MB of archives.
After this operation, 1429 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  libcudnn8-dev 8.6.0.163-1+cuda11.8 [437 MB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  libcudnn8 8.6.0.163-1+cuda11.8 [446 MB]
Fetched 883 MB in 12s (73.3 MB/s)                                              

(Reading database ... 108827 files and directories currently installed.)
Preparing to unpack .../libcudnn8-dev_8.6.0.163-1+cuda11.8_amd64.deb ...
7Progress: [  0%] [..........................................................] 8up

2022-11-17 21:59:00.818664: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 21:59:01.002595: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-17 21:59:02.059397: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local

Thu Nov 17 21:59:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

2022-11-17 21:59:07.087413: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 21:59:07.088428: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 21:59:07.220900: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 21:59:07.221838: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 21:59:07.222693: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

### Metadata

In [2]:
classes = ["Species1", "Species2", "Species3", "Species4", "Species5", "Species6", "Species7", "Species8"]
input_shape = (96, 96, 3)
input_size = input_shape[:-1]
inflation_coeff = 1.5

batch_size = 128 * strategy.num_replicas_in_sync
epochs = 400

print(batch_size)

256


In [3]:
path = os.getcwd()
if not os.path.exists(path+'/training_data_final'):
    shutil.copytree('../input/training-data-final/training_data_final', os.getcwd() + r'/training_data_final')
print(os.listdir(os.getcwd()))

['training_data_final', '.virtual_documents', '__notebook_source__.ipynb']


### Prepare the environment

In [4]:
train_split = 0.8

path = os.getcwd()
if not os.path.exists(path+'/training') and not os.path.exists(path+'/validation'):
    os.mkdir(path+'/training')
    os.mkdir(path+'/validation')

    # Destination path 
    dest_train = path + '/training'
    dest_valid = path + '/validation'

    # Source path
    source = path + '/training_data_final'

    # Create train and validation into the training and validation folders
    for folder in os.listdir(source):
        if not os.path.exists(dest_train + '/' + folder):
            os.mkdir(dest_train + '/' + folder)
        if not os.path.exists(dest_valid + '/' + folder):
            os.mkdir(dest_valid + '/' + folder)
    
        class_source = source + '/' + folder                                                   # Create path of the class
        files = os.listdir(class_source)                                                       # List of files for the class
        random.shuffle(files)                                                                  # Split is performed randomly
        
        # Create training set randomly
        for i in range(int(len(files) * train_split)):
            dest = shutil.copy(class_source+'/'+files[i], dest_train+'/'+folder+'/'+files[i])  # Copy an image in the training set
        
        # Create validation set randomly
        for j in range(i + 1, len(files)):
            dest = shutil.copy(class_source+'/'+files[j], dest_valid+'/'+folder+'/'+files[j])  # copy an image in the validation set

### Preprocessing function

Allows us to simply define a pipeline of preprocessing transformations

In [5]:
from tensorflow.keras.applications.xception import preprocess_input

def preprocessing(image):
    #return tf.image.adjust_saturation(image, 3)
    return preprocess_input(image)
    #return image

### Prepare the training set for standardization

In [6]:
samples = []
targets = []

dest_train = os.getcwd() + '/training'

for folder in os.listdir(dest_train):
    dest_class = dest_train + '/' + folder
    i = int(re.sub("\D", "", folder)) - 1
    for img in os.listdir(dest_class):
        temp = Image.open(dest_class + '/' + img).convert('RGB')
        image = preprocessing(np.squeeze(np.expand_dims(temp, axis=0)))
        label = tfk.utils.to_categorical(i, len(classes))
        samples.append(image)
        targets.append(label)
X_train = np.array(samples)
y_train = np.array(targets, dtype=np.uint8)
print(X_train.shape, X_train.dtype, sep=", ")
print(y_train.shape, y_train.dtype, sep=", ")

# Compute the class weights in order to balance loss during training
y_numeric = []
for v in y_train:
    y_numeric.append(np.argmax(v))

labels = np.unique(np.fromiter([np.argmax(t) for t in y_train], np.int32))
    
class_weights = dict(enumerate(class_weight.compute_class_weight('balanced', classes=labels, y=y_numeric)))
print(class_weights)

(2829, 96, 96, 3), float32
(2829, 8), uint8
{0: 2.389358108108108, 1: 0.8320588235294117, 2: 0.8583131067961165, 3: 0.8667279411764706, 4: 0.8340212264150944, 5: 1.9978813559322033, 6: 0.8243006993006993, 7: 0.8709975369458128}


### Static augmentation (only on training set)

In [7]:
static_aug = True
balanced = False

if static_aug and not os.path.exists(path+'/training_aug'):
    old_train = os.getcwd() + '/training'
    dest_train = os.getcwd() + '/training_aug'
    shutil.copytree(old_train, dest_train)

    desired_amount = int(537 * train_split)

    static_gen = ImageDataGenerator()

    for folder in os.listdir(dest_train):
        dest_path = dest_train + '/' + folder
        label = int(re.sub("\D", "", folder)) - 1

        if balanced:
            to_produce = desired_amount - len(os.listdir(dest_path))
        else:
            class_expansion = [6, 1, 1, 1, 1, 3, 1, 4]
            to_produce = (class_expansion[label] * desired_amount) - len(os.listdir(dest_path))
        
        static_gen_data = static_gen.flow_from_directory(dest_train,
                                                        batch_size=1,
                                                        target_size=input_size,
                                                        classes=[folder],
                                                        class_mode='categorical',      # Targets are directly converted into one-hot vectors
                                                        shuffle=False,
                                                        seed=seed) 

        print(f'Computing {to_produce} augmented images for target "{folder}"')
        os.chdir(dest_path)
        for i in range(0, to_produce):
            Image.fromarray(np.squeeze(next(static_gen_data)[0]).astype(np.uint8)).save(f'aug{i:05}.jpg')
        os.chdir('../')

    os.chdir('../')
    print('\n' + os.getcwd())

Found 406 images belonging to 1 classes.
Computing 1310 augmented images for target "Species8"
Found 177 images belonging to 1 classes.
Computing 1110 augmented images for target "Species6"
Found 429 images belonging to 1 classes.
Computing 0 augmented images for target "Species7"
Found 425 images belonging to 1 classes.
Computing 4 augmented images for target "Species2"
Found 408 images belonging to 1 classes.
Computing 21 augmented images for target "Species4"
Found 148 images belonging to 1 classes.
Computing 2426 augmented images for target "Species1"
Found 412 images belonging to 1 classes.
Computing 17 augmented images for target "Species3"
Found 424 images belonging to 1 classes.
Computing 5 augmented images for target "Species5"

/kaggle/working


### Online augmentation
Lets create the generators we'll need...

In [8]:
shift = 30
train_data_gen = ImageDataGenerator(rotation_range=90,
                                    width_shift_range=shift,
                                    height_shift_range=shift,
                                    horizontal_flip=True,
                                    brightness_range=(0.8, 1.05),
                                    #channel_shift_range=150,
                                    zoom_range=[0.8, 1.3],
                                    shear_range=0.1,
                                    fill_mode='reflect',
                                    preprocessing_function=preprocessing
                                    #featurewise_std_normalization=True,
                                    #featurewise_center=True, 
                                    #rescale=1./255
                                    )

valid_data_gen = ImageDataGenerator(preprocessing_function=preprocessing
                                    #featurewise_std_normalization=True,
                                    #featurewise_center=True, 
                                    #rescale=1./255
                                    )

# Fit the standardization values
#train_data_gen.fit(X_train)
#valid_data_gen.fit(X_train)

... using flow_from_directory

In [9]:
# Setting right paths
path = os.getcwd()
if static_aug:
    training_dir = path + '/training_aug'
else:
    training_dir = path + '/training'
validation_dir = path + '/validation'

# Training
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=batch_size,
                                               target_size=input_size,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=seed)  

# Validation
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               batch_size=batch_size, 
                                               target_size=input_size,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=seed)

# Disable AutoShard
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF

# Create Datasets objects
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float16, tf.uint8),
                                               output_shapes=([None, input_shape[0], input_shape[1], input_shape[2]], [None, len(classes)]))

train_dataset = train_dataset.with_options(options)
train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float16, tf.uint8),
                                               output_shapes=([None, input_shape[0], input_shape[1], input_shape[2]], [None, len(classes)]))

valid_dataset = valid_dataset.with_options(options)
valid_dataset = valid_dataset.repeat()

Found 7722 images belonging to 8 classes.
Found 713 images belonging to 8 classes.


### Prepare the validation set for evaluation purposes

In [10]:
samples = []
targets = []

#mean = train_data_gen.mean
#std = train_data_gen.std

dest_valid = os.getcwd() + '/validation'

for folder in os.listdir(dest_valid):
    dest_class = dest_valid + '/' + folder
    i = int(re.sub("\D", "", folder)) - 1
    for img in os.listdir(dest_class):
        temp = Image.open(dest_class + '/' + img).convert('RGB')
        #image = preprocessing((np.squeeze(np.expand_dims(temp, axis=0)) - mean) / std)
        image = preprocessing(np.squeeze(np.expand_dims(temp, axis=0)))
        label = tfk.utils.to_categorical(i, len(classes))
        samples.append(image)
        targets.append(label)

X_val = np.array(samples, dtype=np.float16)
y_val = np.array(targets, dtype=np.uint8)
print(X_val.shape, X_val.dtype, sep=", ")
print(y_val.shape, y_val.dtype, sep=", ")

(713, 96, 96, 3), float16
(713, 8), uint8


### Models definition functions

In [11]:
def build_tl_model(input_shape):
    tf.random.set_seed(seed)
    
    inflated_dim0 = int(inflation_coeff * input_shape[0])
    inflated_dim1 = int(inflation_coeff * input_shape[1])
    inflated_shape = (inflated_dim0, inflated_dim1, 3)

    # Load the supernet
    supernet = tfk.applications.Xception(include_top=False,
                                         weights="imagenet",
                                         input_shape=inflated_shape)
    
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')
    
    x = tfkl.Resizing(inflated_dim0, inflated_dim1, interpolation="bicubic", name='resizing')(input_layer)
    
    x = supernet(x)

    x = tfkl.GlobalAveragePooling2D(name='gap')(x)
    #x = tfkl.Flatten(name='flatten')(x)

    x = tfkl.Dropout(0.3, seed=seed, name='dropout')(x)
    
    output_layer = tfkl.Dense(
        units = len(classes), 
        activation = 'softmax', 
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'output_layer')(x)
    
    # Connect input and output through the Model class
    model = tfk.Model(inputs = input_layer, outputs = output_layer, name = 'tl_model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(2e-4), metrics='accuracy')

    # Return the model
    return model

### Build the model

In [12]:
with strategy.scope():
    model = build_tl_model(input_shape)
    model.summary()

83683744/83683744 [==============================] - 0s 0us/step
Model: "tl_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 96, 96, 3)]       0         
                                                                 
 resizing (Resizing)         (None, 144, 144, 3)       0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 gap (GlobalAveragePooling2D  (None, 2048)             0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 output_layer (Dense)        (None, 8)                 1639

### Train the model

In [13]:
train_mul = 1

'''
high_period = 5
medium_period = 10
low_period = 5

high_lr = 5e-4  
medium_lr = 1e-4
low_lr = 2e-5

def scheduler(epoch, lr):
    global medium_lr, high_lr

    #convergence factor to shrink high and medium values onto low_lr over time
    tot_period = high_period + medium_period + low_period + medium_period
    if epoch % (tot_period) == (tot_period - 1):
        high_lr = max(high_lr * tf.math.exp(-0.1), low_lr)
        medium_lr = max(medium_lr * tf.math.exp(-0.1), low_lr)

    if epoch % (tot_period) < high_period:
        return high_lr
    elif epoch % (tot_period) < high_period + medium_period:
        return medium_lr    
    elif epoch % (tot_period) < high_period + medium_period + low_period:
        return low_lr    
    return medium_lr
'''

decay_rate = 5  # patience should be set: changes_to_see * decay_rate + 1
min_lr = 2e-5

def scheduler(epoch, lr):
    if epoch % decay_rate == (decay_rate - 1):
        return max(lr * tf.math.exp(-0.1), min_lr)
    return lr

start = time.time()

class ElapsedTimeCallback(tfk.callbacks.Callback):
    def on_test_end(self, epoch, logs=None):
        el = time.time() - start
        print(f'\nElapsed time: {int(el // 60)} minutes {(el % 60):.3f} seconds')

history = model.fit(x=train_dataset,
                    epochs=epochs,                                     # Only indicative since we set "repeat" in training and validation datasets
                    steps_per_epoch=int(len(train_gen) * train_mul),
                    validation_data=valid_dataset,
                    validation_steps=len(valid_gen),
                    class_weight=class_weights,
                    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=21, restore_best_weights=True),
                                 tfk.callbacks.LearningRateScheduler(scheduler),
                                 ElapsedTimeCallback()]
).history

Epoch 1/400


2022-11-17 22:00:11.133040: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2022-11-17 22:00:11.617274: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2022-11-17 22:00:12.775118: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-11-17 22:00:13.863256: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


31/31 [==============================] - ETA: 0s - loss: 1.8707 - accuracy: 0.4628
Elapsed time: 1 minutes 19.922 seconds
31/31 [==============================] - 81s 1s/step - loss: 1.8707 - accuracy: 0.4628 - val_loss: 2.1039 - val_accuracy: 0.2398 - lr: 2.0000e-04
Epoch 2/400
31/31 [==============================] - ETA: 0s - loss: 0.8841 - accuracy: 0.6966
Elapsed time: 1 minutes 53.883 seconds
31/31 [==============================] - 34s 1s/step - loss: 0.8841 - accuracy: 0.6966 - val_loss: 1.6262 - val_accuracy: 0.4670 - lr: 2.0000e-04
Epoch 3/400
31/31 [==============================] - ETA: 0s - loss: 0.5711 - accuracy: 0.8087
Elapsed time: 2 minutes 29.730 seconds
31/31 [==============================] - 36s 1s/step - loss: 0.5711 - accuracy: 0.8087 - val_loss: 1.5818 - val_accuracy: 0.4881 - lr: 2.0000e-04
Epoch 4/400
31/31 [==============================] - ETA: 0s - loss: 0.4112 - accuracy: 0.8592
Elapsed time: 3 minutes 5.830 seconds
31/31 [==============================] 

KeyboardInterrupt: 

### Plot training results

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history['loss'], label='Std training', alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_loss'], label='Std validation', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], label='Std training', alpha=.8, color='#ff7f0e', linestyle='--')
plt.plot(history['val_accuracy'], label='Std validation', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper right')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

### Plot the confusion matrix (evaluated on the validation set)

In [ ]:
predictions = model.predict(X_val)
cm = confusion_matrix(np.argmax(y_val, axis=-1), np.argmax(predictions, axis=-1))

accuracy = accuracy_score(np.argmax(y_val, axis=-1), np.argmax(predictions, axis=-1))
precision = precision_score(np.argmax(y_val, axis=-1), np.argmax(predictions, axis=-1), average='macro')
recall = recall_score(np.argmax(y_val, axis=-1), np.argmax(predictions, axis=-1), average='macro')
f1 = f1_score(np.argmax(y_val, axis=-1), np.argmax(predictions, axis=-1), average=None)
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

plt.figure(figsize=(10,8))
sns.heatmap(cm.T, xticklabels=classes, yticklabels=classes)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

### Save the model

In [14]:
model.save('best_model', include_optimizer=False)

shutil.make_archive('best_model', 'zip', 'best_model')
FileLink(r'best_model.zip')

/kaggle/working/best_model.zip

### Improve the classifier module of the previously trained supernet

In [23]:
def build_classifier_model(input_shape):
    tf.random.set_seed(seed)
    
    inflated_dim0 = int(inflation_coeff * input_shape[0])
    inflated_dim1 = int(inflation_coeff * input_shape[1])
    inflated_shape = (inflated_dim0, inflated_dim1, 3)

    # Load the supernet
    supernet = tfk.applications.Xception(include_top=False,
                                         input_shape=inflated_shape)

    # Recover previous weights
    supernet.set_weights(tfk.models.load_model('best_model').get_layer('xception').get_weights())
    
    # Use the supernet only as feature extractor
    supernet.trainable = False  # "True" for fine tuning
    for i, layer in enumerate(supernet.layers[:-25]):
      layer.trainable = False
      #print(i, layer.name, layer.trainable)
    
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')
    
    x = tfkl.Resizing(inflated_dim0, inflated_dim1, interpolation="bicubic", name='resizing')(input_layer)
    
    x = supernet(x)

    x = tfkl.GlobalAveragePooling2D(name='gap')(x)
    
    x_gap = x

    x = tfkl.Dropout(0.3, seed=seed, name='dropout')(x)

    x = tfkl.Dense(
        units = 2048,  
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name = 'classifier')(x)

    # Skip connection
    x = tfkl.Add(name='adder')([x_gap, x])
    
    output_layer = tfkl.Dense(
                   units = len(classes), 
                   activation = 'softmax', 
                   kernel_initializer = tfk.initializers.GlorotUniform(seed),
                   name = 'output_layer')(x)
    
    # Connect input and output through the Model class
    model = tfk.Model(inputs = input_layer, outputs = output_layer, name = 'classifier_model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(3e-4), metrics='accuracy')

    # Return the model
    return model

In [24]:
with strategy.scope():
    classifier_model = build_classifier_model(input_shape)
    classifier_model.summary()

Model: "classifier_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 resizing (Resizing)            (None, 144, 144, 3)  0           ['input_layer[0][0]']            
                                                                                                  
 xception (Functional)          (None, 5, 5, 2048)   20861480    ['resizing[0][0]']               
                                                                                                  
 gap (GlobalAveragePooling2D)   (None, 2048)         0           ['xception[0][0]']               
                                                                                   

In [25]:
train_mul = 1

start = time.time()

classifier_history = classifier_model.fit(x=train_dataset,
                                          epochs=epochs,                                  
                                          steps_per_epoch=int(len(train_gen) * train_mul),
                                          validation_data=valid_dataset,
                                          validation_steps=len(valid_gen),
                                          class_weight=class_weights,
                                          callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=26, restore_best_weights=True),
                                                       tfk.callbacks.LearningRateScheduler(scheduler),
                                                       ElapsedTimeCallback()]
).history

Epoch 1/400
31/31 [==============================] - ETA: 0s - loss: 0.6699 - accuracy: 0.7919
Elapsed time: 0 minutes 47.277 seconds


2022-11-17 22:12:29.813868: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16777216 exceeds 10% of free system memory.


31/31 [==============================] - 48s 1s/step - loss: 0.6699 - accuracy: 0.7919 - val_loss: 0.8404 - val_accuracy: 0.7209 - lr: 3.0000e-04
Epoch 2/400


2022-11-17 22:12:30.296165: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16777216 exceeds 10% of free system memory.


29/31 [===========================>..] - ETA: 1s - loss: 0.4550 - accuracy: 0.8473

KeyboardInterrupt: 

### Plot training results

In [ ]:
history = classifier_history

plt.figure(figsize=(15,5))
plt.plot(history['loss'], label='Std training', alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_loss'], label='Std validation', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], label='Std training', alpha=.8, color='#ff7f0e', linestyle='--')
plt.plot(history['val_accuracy'], label='Std validation', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper right')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

### Plot the confusion matrix (evaluated on the validation set)

In [ ]:
predictions = classifier_model.predict(X_val)
cm = confusion_matrix(np.argmax(y_val, axis=-1), np.argmax(predictions, axis=-1))

accuracy = accuracy_score(np.argmax(y_val, axis=-1), np.argmax(predictions, axis=-1))
precision = precision_score(np.argmax(y_val, axis=-1), np.argmax(predictions, axis=-1), average='macro')
recall = recall_score(np.argmax(y_val, axis=-1), np.argmax(predictions, axis=-1), average='macro')
f1 = f1_score(np.argmax(y_val, axis=-1), np.argmax(predictions, axis=-1), average=None)
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

plt.figure(figsize=(10,8))
sns.heatmap(cm.T, xticklabels=classes, yticklabels=classes)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

### Save the model

In [ ]:
classifier_model.save('best_model_improved', include_optimizer=False)

shutil.make_archive('best_model_improved', 'zip', 'best_model_improved')
FileLink(r'best_model_improved.zip')